In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch

In [2]:
df=pd.read_csv('diabetes.csv')
df=df.drop(columns='family_histroy')
df.replace(4.860753, np.nan, inplace=True)
df.dropna(inplace=True)
df["smoking"] = df["smoking"].replace({1.0: 0, 2.0: 0, 3.0: 1})
df_copy=df
df_copy["smoking"] = df["smoking"].astype(int)


X = df_copy.drop('smoking', axis=1)
y = df_copy['smoking']

In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


X_train_np = X_train.astype(np.float32)
X_test_np = X_test.astype(np.float32)
y_train_np = y_train.values.astype(int)
y_test_np = y_test.values.astype(int)


tabnet_model = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.001),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
)

tabnet_model.fit(
    X_train=X_train_np, y_train=y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=['accuracy'],
    max_epochs=50,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
)


y_pred = tabnet_model.predict(X_test_np)
acc = accuracy_score(y_test_np, y_pred)
print(f"\n TabNet Test Accuracy: {acc:.4f}")
print(classification_report(y_test_np, y_pred))

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.94945 | val_0_accuracy: 0.30982 |  0:00:00s
epoch 10 | loss: 0.75377 | val_0_accuracy: 0.49693 |  0:00:00s
epoch 20 | loss: 0.66026 | val_0_accuracy: 0.60736 |  0:00:00s
epoch 30 | loss: 0.61046 | val_0_accuracy: 0.64571 |  0:00:01s
epoch 40 | loss: 0.57055 | val_0_accuracy: 0.67178 |  0:00:01s
Stop training because you reached max_epochs = 50 with best_epoch = 49 and best_val_0_accuracy = 0.67945

 TabNet Test Accuracy: 0.6794
              precision    recall  f1-score   support

           0       0.35      0.12      0.18       189
           1       0.72      0.91      0.80       463

    accuracy                           0.68       652
   macro avg       0.53      0.51      0.49       652
weighted avg       0.61      0.68      0.62       652



/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
